In [1]:
import rebound
%matplotlib inline
import matplotlib.pyplot as plt
import observations
import state
import mcmc
import numpy as np

/Users/rein/git/rebound/venv/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
true_state = state.State(planets=[{"m":1e-3, "a":1., "h":0.2}])
obs = observations.FakeObservation(true_state, Npoints=100, error=1e-4, tmax=10.)

In [3]:
#fig = plt.figure(figsize=(10,5))
#ax = plt.subplot(111)
#times, rv = true_state.get_rv_plotting(tmax=10.)
#ax.plot(times, rv)
#ax.plot(obs.t, obs.rv, ".")

In [4]:
varState = state.StateVar(planets=[{"m":1e-3, "a":1., "h":0.2, "k":0., "l":1.4}], ignore_vars=["a","m"])

varState.get_chi2(obs)

92.622928598526983

In [5]:
Delta = 1e-4
s = state.StateVar(planets=[{"m":1e-3, "a":1., "h":0.2, "k":0., "l":1.4}], ignore_vars=["a","m"])
chi2 = s.get_chi2(obs)
fd_d = np.zeros(s.Nvars)
fd_dd = np.zeros((s.Nvars,s.Nvars))
for i in range(s.Nvars):
    shift = np.zeros(s.Nvars)
    shift[i] = Delta
    s.shift(shift);
    chi2p = s.get_chi2(obs)
    s.shift(-shift);
    fd_d[i] = (chi2p-chi2)/Delta
    for j in range(s.Nvars):
        shift = np.zeros(s.Nvars)
        shift[i] += Delta
        shift[j] += Delta
        s.shift(shift);
        chi2pp = s.get_chi2(obs)
        s.shift(-shift);
        
        shift = np.zeros(s.Nvars)
        shift[i] -= Delta
        shift[j] += Delta
        s.shift(shift);
        chi2mp = s.get_chi2(obs)
        s.shift(-shift);
        
        shift = np.zeros(s.Nvars)
        shift[i] += Delta
        shift[j] -= Delta
        s.shift(shift);
        chi2pm = s.get_chi2(obs)
        s.shift(-shift);
        
        shift = np.zeros(s.Nvars)
        shift[i] -= Delta
        shift[j] -= Delta
        s.shift(shift);
        chi2mm = s.get_chi2(obs)
        s.shift(-shift);
        
        fd_dd[i][j] = (chi2pp-chi2mp-chi2pm+chi2mm)/(4.*Delta**2)

chi2, v_d, v_dd = s.get_chi2_d_dd(obs)

print("First order:")
print(fd_d)
print(v_d)
print("\nSecond order:")
print(fd_dd)
print(v_dd)



First order:
[ 33.98108205 -38.35868678  80.80293712]
[ 33.98392686 -38.35948876  80.8047794 ]

Second order:
[[-56.88327995 -26.87977307 -69.24770943]
 [-26.87977307  16.02522524 -84.47840187]
 [-69.24770943 -84.47840187 -36.84355825]]
[[-56.88326162 -26.87976424 -69.24771385]
 [-26.87976424  16.02523879 -84.47839284]
 [-69.24771385 -84.47839284 -36.84355767]]


In [9]:
mh = mcmc.Mh(s,obs)
mh.scales = 1e-3*np.ones(s.Nvars)
mh.generate_proposal().planets

[{'a': 1.0,
  'h': 0.20203361574567361,
  'k': 0.00043313013576109762,
  'l': 1.3993607125183578,
  'm': 0.001}]